In [1]:
import pandas as pd

In [367]:
hometown = pd.read_csv("C:/Users/tanne/Documents/SP 24/Sports/Project/hometown.csv")
fights = pd.read_csv("C:/Users/tanne/Documents/SP 24/Sports/Project/location_result.csv")

In [368]:
hometown.columns

Index(['BPrev', 'BStreak', 'B_Age', 'B_Height', 'B_HomeTown', 'B_ID',
       'B_Location', 'B_Name', 'B_Weight',
       'B__Round1_Grappling_Reversals_Landed',
       ...
       'R__Round5_TIP_Ground Time', 'R__Round5_TIP_Guard Control Time',
       'R__Round5_TIP_Half Guard Control Time',
       'R__Round5_TIP_Misc. Ground Control Time',
       'R__Round5_TIP_Mount Control Time', 'R__Round5_TIP_Neutral Time',
       'R__Round5_TIP_Side Control Time', 'R__Round5_TIP_Standing Time',
       'winby', 'winner'],
      dtype='object', length=894)

In [369]:
hometown_blue = hometown[['B_Name', 'B_HomeTown', 'B_Location']].rename(columns = {'B_Name': 'name',
                                                                        'B_HomeTown': 'hometown',
                                                                        "B_Location": 'location'})
hometown_red = hometown[['R_Name', 'R_HomeTown', 'R_Location']].rename(columns = {'R_Name': 'name',
                                                                        'R_HomeTown': 'hometown',
                                                                        "R_Location": 'location'})

In [370]:
f = pd.concat([hometown_blue, hometown_red], ignore_index=True)

In [371]:
fighters = f.groupby(["name", "location"])
fighters.head(5)

,name,hometown,location
0,Alistair Overeem,Hounslow England,Amsterdam The Netherlands
1,Ricardo Lamas,"Chicago, Illinois United States","Chicago, Illinois United States"
2,Urijah Faber,"Isla Vista , California USA","Sacramento, California USA"
3,Danny Martinez,"San Diego, CA USA","San Diego, CA USA"
4,Tom Watson,Southampton England,Southampton England
...,...,...,...
4618,Geoff Neal,"Austin, Texas United States","Dallas, Texas United States"
4621,Abdul Razak Alhassan,Accra Ghana,"Fort Worth, Texas United States"
4623,Irene Aldana,"Culiacan, Sinaloa Mexico","Gudalajara, Jalisco Mexico"
4626,Lina Lansberg,Karlstad Sweden,Malmo Sweden


In [372]:
grouped_df = f.groupby('name').agg({
    'hometown': 'first',
    'location': lambda x: list(set(x))  # Aggregating training locations into a list
}).reset_index()

In [373]:
grouped_df.rename(columns={'location': 'training_location'}, inplace=True)
grouped_df

,name,hometown,training_location
0,Aaron Phillips,"Houston, TX USA","[Lafayette, LA USA]"
1,Abdul Razak Alhassan,Accra Ghana,"[Fort Worth, Texas United States]"
2,Abdul-Kerim Edilov,Chechen Republic Russia,[Grozny Russia]
3,Abel Trujillo,"Greensboro, North Carolina United States","[Boca Raton, Florida United States]"
4,Abner Lloveras,Barcelona Spain,[Barcelona Spain]
...,...,...,...
1094,Zak Cummings,"Irving, Texas United States","[Kansas City, Missouri United States]"
1095,Zak Ottow,"Milwaukee, Wisconsin United States","[Milwaukee, Wisconsin United States]"
1096,Zhang Lipeng,Inner Mongolia Province P.R. China,[Beijing China]
1097,Zu Anyanwu,"Trenton, New Jersey United States","[Langhorne, Pennsylvania ]"


In [374]:
fights.columns

Index(['url', 'event_title', 'organisation', 'date', 'location', 'match_nr',
       'fighter1_url', 'fighter2_url', 'fighter1_name', 'fighter2_name',
       'fighter1_result', 'fighter2_result', 'win_method', 'win_details',
       'referee', 'round', 'time'],
      dtype='object')

In [375]:
fights = pd.merge(fights, grouped_df[['name', 'hometown', 'training_location']].rename(columns={
    'name': 'fighter1_name',
    'hometown': 'fighter1_hometown',
    'training_location': 'fighter1_training_location'
}), how='left', on='fighter1_name')

# No need to drop 'name' here since we're not merging it in directly.

# Merge for 'fighter2_name', similarly renaming columns to avoid duplication
fights = pd.merge(fights, grouped_df[['name', 'hometown', 'training_location']].rename(columns={
    'name': 'fighter2_name',
    'hometown': 'fighter2_hometown',
    'training_location': 'fighter2_training_location'
}), how='left', on='fighter2_name')

# After these operations, there shouldn't be any undesired duplicated columns, assuming all renames were accurate.

# Selecting the specified columns for the final DataFrame, adjusted if necessary


In [376]:
final_df = fights[[
    'fighter1_name', 'fighter1_hometown', 'fighter1_training_location', 
    'fighter2_name', 'fighter2_hometown', 'fighter2_training_location', 'fighter1_result', 'fighter2_result',
    'win_method', 'win_details', 'location'
]]

final_df

,fighter1_name,fighter1_hometown,fighter1_training_location,fighter2_name,fighter2_hometown,fighter2_training_location,fighter1_result,fighter2_result,win_method,win_details,location
0,Ciryl Gane,NaN,NaN,Derrick Lewis,"New Orleans, Louisiana United States","[Houston, Texas United States]",win,loss,TKO,Punches,"Toyota Center, Houston, Texas, United States"
1,Jose Aldo,Manaus Brazil,[Rio de Janeiro Brazil],Pedro Munhoz,Sao Paulo Brazil,[Sao Paulo Brazil],win,loss,Decision,Unanimous,"Toyota Center, Houston, Texas, United States"
2,Vicente Luque,"Westwood, New Jersey United States",[Brasília Brasil],Michael Chiesa,"Aurora, Colorado United States","[Spokane Valley, Washington United States]",win,loss,Submission,Brabo Choke,"Toyota Center, Houston, Texas, United States"
3,Tecia Torres,"Fall River, Massachusetts United States","[Colorado Springs, Colorado United States]",Angela Hill,"Clinton, Maryland United States","[San Diego, California United States]",win,loss,Decision,Unanimous,"Toyota Center, Houston, Texas, United States"
4,Yadong Song,NaN,NaN,Casey Kenney,"Portland, Indiana United States","[Tuscan, Arizona United States]",win,loss,Decision,Split,"Toyota Center, Houston, Texas, United States"
...,...,...,...,...,...,...,...,...,...,...,...
10443,Eddie Ng,NaN,NaN,Chunbo Yuan,NaN,NaN,win,loss,KO,Punches,"Singapore Indoor Stadium, Kallang, Singapore, ..."
10444,Radeem Rahman,NaN,NaN,Susovan Ghosh,NaN,NaN,win,loss,TKO,Punches,"Singapore Indoor Stadium, Kallang, Singapore, ..."
10445,Vuyisile Colossa,NaN,NaN,Xing Yu Ma,NaN,NaN,win,loss,TKO,Punches,"Singapore Indoor Stadium, Kallang, Singapore, ..."
10446,Yodsanan Sityodtong,NaN,NaN,Daniel Mashamaite,NaN,NaN,win,loss,TKO,Knee,"Singapore Indoor Stadium, Kallang, Singapore, ..."


In [377]:
import pandas as pd

# Sample structure for 'final_df' based on your description
# Assuming 'final_df' has these columns: ['fighter1_name', 'fighter1_hometown', 'fighter1_training_location',
# 'fighter2_name', 'fighter2_hometown', 'fighter2_training_location', 'win_method', 'win_details', 'location',
# 'fighter1_result', 'fighter2_result']

# Splitting 'final_df' into two parts, one for each fighter's perspective
fighter1_df = final_df[['fighter1_name', 'fighter1_hometown', 'fighter1_training_location', 'fighter1_result', 'win_method', 'win_details', 'location']].rename(columns={
    'fighter1_name': 'fighter name',
    'fighter1_hometown': 'hometown',
    'fighter1_training_location': 'training_location',
    'fighter1_result': 'result'
})

fighter2_df = final_df[['fighter2_name', 'fighter2_hometown', 'fighter2_training_location', 'fighter2_result', 'win_method', 'win_details', 'location']].rename(columns={
    'fighter2_name': 'fighter name',
    'fighter2_hometown': 'hometown',
    'fighter2_training_location': 'training_location',
    'fighter2_result': 'result'
})

# Concatenating the two DataFrames to have two rows per fight
new_df = pd.concat([fighter1_df, fighter2_df], ignore_index=True)

# Display the first few rows to verify the structure
new_df.head(5)

,fighter name,hometown,training_location,result,win_method,win_details,location
0,Ciryl Gane,NaN,NaN,win,TKO,Punches,"Toyota Center, Houston, Texas, United States"
1,Jose Aldo,Manaus Brazil,[Rio de Janeiro Brazil],win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States"
2,Vicente Luque,"Westwood, New Jersey United States",[Brasília Brasil],win,Submission,Brabo Choke,"Toyota Center, Houston, Texas, United States"
3,Tecia Torres,"Fall River, Massachusetts United States","[Colorado Springs, Colorado United States]",win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States"
4,Yadong Song,NaN,NaN,win,Decision,Split,"Toyota Center, Houston, Texas, United States"


In [378]:
# Remove rows with any NA values across all columns
clean_df = new_df.dropna()

# Display the cleaned DataFrame to verify no rows have NA values
clean_df

,fighter name,hometown,training_location,result,win_method,win_details,location
1,Jose Aldo,Manaus Brazil,[Rio de Janeiro Brazil],win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States"
2,Vicente Luque,"Westwood, New Jersey United States",[Brasília Brasil],win,Submission,Brabo Choke,"Toyota Center, Houston, Texas, United States"
3,Tecia Torres,"Fall River, Massachusetts United States","[Colorado Springs, Colorado United States]",win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States"
7,Alonzo Menifield,"Los Angeles, California United States","[Dallas, Texas United States]",win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States"
8,Jessica Penne,"Newport Beach, California United States","[Huntington Beach, California United States]",win,Submission,Armbar,"Toyota Center, Houston, Texas, United States"
...,...,...,...,...,...,...,...
20085,Kazuki Tokudome,Tokyo Japan,[Tokyo Japan],loss,TKO,Punches,"Singapore Indoor Stadium, Kallang, Singapore"
20097,Daichi Abe,Niigata Prefecture Japan,[Tokyo Japan],loss,KO,Body Kick,"Thuwunna Youth Training Centre Stadium, Yangon..."
20118,Leandro Issa,"Ubatuba, Sao Paulo Brazil",[Singapore],loss,KO,Punch,"Impact Arena, Bangkok, Thailand"
20163,Kazuki Tokudome,Tokyo Japan,[Tokyo Japan],loss,Decision,Unanimous,"Putra Indoor Stadium, Kuala Lumpur, Malaysia"


In [379]:
newer_df = clean_df

In [380]:
## CREATE EVENT COLUMNS

def parse_location(location_str):
    parts = location_str.split(', ')
    # Initialize default values
    country = None
    state = None
    city = None
    
    # Check the number of parts and assign values accordingly
    if len(parts) >= 3:  # Expected format for USA locations or locations with city, state, and country
        country = parts[-1]
        state = parts[-2] if country == 'United States' else None
        city = parts[-3] if country == 'United States' else parts[-2]
    elif len(parts) == 2:  # Expected format for non-USA locations without a state
        country = parts[-1]
        city = parts[-2]
    elif len(parts) == 1:  # Only country is present
        country = parts[0]
    
    return city, state, country

# Then apply this function to your DataFrame as before:
newer_df[['event_city', 'event_state', 'event_country']] = newer_df.apply(lambda row: pd.Series(parse_location(row['location'])), axis=1)

C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\2917083753.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newer_df[['event_city', 'event_state', 'event_country']] = newer_df.apply(lambda row: pd.Series(parse_location(row['location'])), axis=1)
C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\2917083753.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newer_df[['event_city', 'event_state', 'event_country']] = newer_df.apply(lambda row: pd.Series(parse_location(row['location'])), axis=1)
C:\Users

In [386]:
## CREATE HOMETOWN COLUMNS


import pycountry
import pandas as pd

# Assuming newer_df is already defined and contains a 'hometown' column.

# Generate a list of all country names, with special handling for the United States
country_names = [country.name for country in pycountry.countries if country.name not in ["United States", "USA"]]

def parse_hometown(hometown_str, country_names):
    city = None
    state = None
    country = None

    # Check both "United States" and "USA"
    if 'United States' in hometown_str or 'USA' in hometown_str:
        country = 'United States'  # Standardize to "United States"
        # Remove both possible country names for further parsing
        city_state_str = hometown_str.replace('United States', '').replace('USA', '').strip()
        if ', ' in city_state_str:
            parts = city_state_str.rsplit(', ', 1)
            if len(parts) == 2:
                city, state = parts
            else:
                city = parts[0]
        else:
            city = city_state_str  # Handle case without comma
    else:
        # For international locations: Check against the country names list
        for country_name in country_names:
            if hometown_str.endswith(country_name):
                country = country_name
                city = hometown_str[:-len(country_name)].strip()
                break

    # Handling cases where no country match was found in the list
    if not country:
        city = hometown_str  # Default fallback if no country is matched

    return city, state, country

# Apply the parsing function to the 'hometown' column in newer_df
newer_df[['hometown_city', 'hometown_state', 'hometown_country']] = newer_df['hometown'].apply(
    lambda x: pd.Series(parse_hometown(x, country_names))
)

# Display the updated DataFrame
print(newer_df[['hometown', 'hometown_city', 'hometown_state', 'hometown_country']])


                                      hometown       hometown_city  \
1                                Manaus Brazil              Manaus   
2           Westwood, New Jersey United States            Westwood   
3      Fall River, Massachusetts United States          Fall River   
7       Los Angeles, California United States          Los Angeles   
8      Newport Beach, California United States       Newport Beach   
...                                        ...                 ...   
20085                              Tokyo Japan               Tokyo   
20097                 Niigata Prefecture Japan  Niigata Prefecture   
20118                Ubatuba, Sao Paulo Brazil  Ubatuba, Sao Paulo   
20163                              Tokyo Japan               Tokyo   
20838                Ubatuba, Sao Paulo Brazil  Ubatuba, Sao Paulo   

      hometown_state hometown_country  
1               None           Brazil  
2         New Jersey    United States  
3      Massachusetts    United States  

C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\567808878.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newer_df[['hometown_city', 'hometown_state', 'hometown_country']] = newer_df['hometown'].apply(
C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\567808878.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newer_df[['hometown_city', 'hometown_state', 'hometown_country']] = newer_df['hometown'].apply(
C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\567808878.py:45: SettingWithCopyWarning: 
A 

In [387]:
newer_df.tail(10)

,fighter name,hometown,training_location,result,win_method,win_details,location,event_city,event_state,event_country,training_city,training_state,training_country,hometown_city,hometown_state,hometown_country
19873,Daichi Abe,Niigata Prefecture Japan,Tokyo Japan,loss,Submission,Rear-Naked Choke,"Axiata Arena, Kuala Lumpur, Malaysia",Kuala Lumpur,None,Malaysia,Tokyo Japan,NaN,NaN,Niigata Prefecture,None,Japan
19897,Yoshihiro Akiyama,Osaka Japan,nan,loss,Decision,Unanimous,"Shanghai, China",Shanghai,None,China,nan,NaN,NaN,Osaka,None,Japan
19915,Sage Northcutt,"Houston, Texas United States","Katy, Texas United States",loss,KO,Punch,"Singapore Indoor Stadium, Kallang, Singapore",Kallang,None,Singapore,Katy,Texas,United States,Houston,Texas,United States
19929,Yushin Okami,Kanagawa Japan,Kanagawa Japan,loss,TKO,Punches,"Istora Senayan, Jakarta, Indonesia",Jakarta,None,Indonesia,Kanagawa Japan,NaN,NaN,Kanagawa,None,Japan
19976,Eddie Alvarez,"Philadelphia, Pennsylvania United States","Philadelphia, Pennsylvania United States",loss,TKO,Punches,"Ryogoku Kokugikan, Tokyo, Japan",Tokyo,None,Japan,Philadelphia,Pennsylvania,United States,Philadelphia,Pennsylvania,United States
20085,Kazuki Tokudome,Tokyo Japan,Tokyo Japan,loss,TKO,Punches,"Singapore Indoor Stadium, Kallang, Singapore",Kallang,None,Singapore,Tokyo Japan,NaN,NaN,Tokyo,None,Japan
20097,Daichi Abe,Niigata Prefecture Japan,Tokyo Japan,loss,KO,Body Kick,"Thuwunna Youth Training Centre Stadium, Yangon...",Yangon,None,Myanmar,Tokyo Japan,NaN,NaN,Niigata Prefecture,None,Japan
20118,Leandro Issa,"Ubatuba, Sao Paulo Brazil",Singapore,loss,KO,Punch,"Impact Arena, Bangkok, Thailand",Bangkok,None,Thailand,Singapore,NaN,NaN,"Ubatuba, Sao Paulo",None,Brazil
20163,Kazuki Tokudome,Tokyo Japan,Tokyo Japan,loss,Decision,Unanimous,"Putra Indoor Stadium, Kuala Lumpur, Malaysia",Kuala Lumpur,None,Malaysia,Tokyo Japan,NaN,NaN,Tokyo,None,Japan
20838,Leandro Issa,"Ubatuba, Sao Paulo Brazil",Singapore,loss,TKO,Punches,"Singapore Indoor Stadium, Kallang, Singapore, ...",Singapore,None,Singapore,Singapore,NaN,NaN,"Ubatuba, Sao Paulo",None,Brazil


In [383]:
## CONVERT TRAINING LOCATION TO STRING

# Convert 'training_location' to string
newer_df['training_location'] = newer_df['training_location'].astype(str)

# Now attempt to remove the first and last characters (brackets)
newer_df['training_location'] = newer_df['training_location'].apply(
    lambda x: x[1:-1] if x.startswith('[') and x.endswith(']') else x
)

# Remove leading and trailing single quotation marks from 'training_location'
newer_df['training_location'] = newer_df['training_location'].apply(
    lambda x: x[1:-1] if x.startswith("'") and x.endswith("'") else x
)

# Display the first 5 rows to verify the changes
newer_df.head(5)


C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\3869205021.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newer_df['training_location'] = newer_df['training_location'].astype(str)
C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\3869205021.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newer_df['training_location'] = newer_df['training_location'].apply(
C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\3869205021.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,fighter name,hometown,training_location,result,win_method,win_details,location,event_city,event_state,event_country
1,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States
2,Vicente Luque,"Westwood, New Jersey United States",Brasília Brasil,win,Submission,Brabo Choke,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States
3,Tecia Torres,"Fall River, Massachusetts United States","Colorado Springs, Colorado United States",win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States
7,Alonzo Menifield,"Los Angeles, California United States","Dallas, Texas United States",win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States
8,Jessica Penne,"Newport Beach, California United States","Huntington Beach, California United States",win,Submission,Armbar,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States


In [426]:
import pycountry
import pandas as pd
import numpy as np

# Generate a list of all country names, with special handling for the United States
country_names = [country.name for country in pycountry.countries if country.name not in ["United States", "USA"]]

def parse_training_location(location_str, country_names):
    city = None
    state = None
    country = None

    # Check both "United States" and "USA"
    if 'United States' in location_str or 'USA' in location_str:
        country = 'United States'  # Standardize to "United States"
        # Remove both possible country names for further parsing
        city_state_str = location_str.replace('United States', '').replace('USA', '').strip()
        if ', ' in city_state_str:
            parts = city_state_str.rsplit(', ', 1)
            if len(parts) == 2:
                city, state = parts
            else:
                city = parts[0]
        else:
            city = city_state_str  # Handle case without comma
    else:
        # For international locations: Check against the country names list
        for country_name in country_names:
            if location_str.endswith(country_name):
                country = country_name
                city = location_str[:-len(country_name)].strip()
                break

    # Handling cases where no country match was found in the list
    if not country:
        city = location_str  # Default fallback if no country is matched

    return city, state, country

# Assuming newer_df is your DataFrame and it contains 'training_location' after brackets have been removed
# Apply the parsing function to the 'training_location' column in newer_df
newer_df[['training_city', 'training_state', 'training_country']] = newer_df['training_location'].apply(
    lambda x: pd.Series(parse_training_location(x, country_names))
)

# Display the updated DataFrame
print(newer_df[['training_location', 'training_city', 'training_state', 'training_country']])


                                training_location     training_city  \
1                           Rio de Janeiro Brazil    Rio de Janeiro   
2                                 Brasília Brasil   Brasília Brasil   
3        Colorado Springs, Colorado United States  Colorado Springs   
7                     Dallas, Texas United States            Dallas   
8      Huntington Beach, California United States  Huntington Beach   
...                                           ...               ...   
20085                                 Tokyo Japan             Tokyo   
20097                                 Tokyo Japan             Tokyo   
20118                                   Singapore                     
20163                                 Tokyo Japan             Tokyo   
20838                                   Singapore                     

      training_state training_country  
1               None           Brazil  
2               None             None  
3           Colorado    Uni

C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\1786337096.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newer_df[['training_city', 'training_state', 'training_country']] = newer_df['training_location'].apply(


In [427]:
newer_df.tail(10)

,fighter name,hometown,training_location,result,win_method,win_details,location,event_city,event_state,event_country,training_city,training_state,training_country,hometown_city,hometown_state,hometown_country
19873,Daichi Abe,Niigata Prefecture Japan,Tokyo Japan,loss,Submission,Rear-Naked Choke,"Axiata Arena, Kuala Lumpur, Malaysia",Kuala Lumpur,None,Malaysia,Tokyo,None,Japan,Niigata Prefecture,None,Japan
19897,Yoshihiro Akiyama,Osaka Japan,nan,loss,Decision,Unanimous,"Shanghai, China",Shanghai,None,China,nan,None,None,Osaka,None,Japan
19915,Sage Northcutt,"Houston, Texas United States","Katy, Texas United States",loss,KO,Punch,"Singapore Indoor Stadium, Kallang, Singapore",Kallang,None,Singapore,Katy,Texas,United States,Houston,Texas,United States
19929,Yushin Okami,Kanagawa Japan,Kanagawa Japan,loss,TKO,Punches,"Istora Senayan, Jakarta, Indonesia",Jakarta,None,Indonesia,Kanagawa,None,Japan,Kanagawa,None,Japan
19976,Eddie Alvarez,"Philadelphia, Pennsylvania United States","Philadelphia, Pennsylvania United States",loss,TKO,Punches,"Ryogoku Kokugikan, Tokyo, Japan",Tokyo,None,Japan,Philadelphia,Pennsylvania,United States,Philadelphia,Pennsylvania,United States
20085,Kazuki Tokudome,Tokyo Japan,Tokyo Japan,loss,TKO,Punches,"Singapore Indoor Stadium, Kallang, Singapore",Kallang,None,Singapore,Tokyo,None,Japan,Tokyo,None,Japan
20097,Daichi Abe,Niigata Prefecture Japan,Tokyo Japan,loss,KO,Body Kick,"Thuwunna Youth Training Centre Stadium, Yangon...",Yangon,None,Myanmar,Tokyo,None,Japan,Niigata Prefecture,None,Japan
20118,Leandro Issa,"Ubatuba, Sao Paulo Brazil",Singapore,loss,KO,Punch,"Impact Arena, Bangkok, Thailand",Bangkok,None,Thailand,,None,Singapore,"Ubatuba, Sao Paulo",None,Brazil
20163,Kazuki Tokudome,Tokyo Japan,Tokyo Japan,loss,Decision,Unanimous,"Putra Indoor Stadium, Kuala Lumpur, Malaysia",Kuala Lumpur,None,Malaysia,Tokyo,None,Japan,Tokyo,None,Japan
20838,Leandro Issa,"Ubatuba, Sao Paulo Brazil",Singapore,loss,TKO,Punches,"Singapore Indoor Stadium, Kallang, Singapore, ...",Singapore,None,Singapore,,None,Singapore,"Ubatuba, Sao Paulo",None,Brazil


# Splits

In [428]:
splits = newer_df[newer_df['win_method'] == 'Decision']
splits

,fighter name,hometown,training_location,result,win_method,win_details,location,event_city,event_state,event_country,training_city,training_state,training_country,hometown_city,hometown_state,hometown_country
1,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,Rio de Janeiro,None,Brazil,Manaus,None,Brazil
3,Tecia Torres,"Fall River, Massachusetts United States","Colorado Springs, Colorado United States",win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,Colorado Springs,Colorado,United States,Fall River,Massachusetts,United States
7,Alonzo Menifield,"Los Angeles, California United States","Dallas, Texas United States",win,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,Dallas,Texas,United States,Los Angeles,California,United States
18,Chris Gruetzemacher,"Page, Arizona United States","New Bern, North Carolina United States",win,Decision,Unanimous,"UFC Apex, Las Vegas, Nevada, United States",Las Vegas,Nevada,United States,New Bern,North Carolina,United States,Page,Arizona,United States
26,Maycee Barber,"Greeley, Colorado United States","Fort Collins, Colorado United States",win,Decision,Split,"UFC Apex, Las Vegas, Nevada, United States",Las Vegas,Nevada,United States,Fort Collins,Colorado,United States,Greeley,Colorado,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19515,Daniel Pineda,"Dallas, TX USA","Houston, TX USA",loss,Decision,Unanimous,"Central Pavilion Arena, Robstown, Texas, Unite...",Robstown,Texas,United States,Houston,TX,United States,Dallas,TX,United States
19573,Eddie Alvarez,"Philadelphia, Pennsylvania United States","Philadelphia, Pennsylvania United States",loss,Decision,Unanimous,"Singapore Indoor Stadium, Singapore",Singapore Indoor Stadium,None,Singapore,Philadelphia,Pennsylvania,United States,Philadelphia,Pennsylvania,United States
19800,Hernani Perpetuo,Rio de Janeiro Brazil,Rio de Janeiro Brazil,loss,Decision,Unanimous,"Ryogoku Kokugikan, Tokyo, Japan",Tokyo,None,Japan,Rio de Janeiro,None,Brazil,Rio de Janeiro,None,Brazil
19863,Yushin Okami,Kanagawa Japan,Kanagawa Japan,loss,Decision,Unanimous,"Mall Of Asia Arena, Manila, Philippines",Manila,None,Philippines,Kanagawa,None,Japan,Kanagawa,None,Japan


In [429]:
matches = splits.apply(lambda row: row['event_city'] == row['hometown_city'], axis=1).sum()

print(f"Number of instances where 'event_city' and 'hometown_city' match: {matches}")

# Calculate the number of instances where 'event_state' and 'hometown_state' match
state_matches = splits.apply(lambda row: row['event_state'] == row['hometown_state'], axis=1).sum()

print(f"Number of instances where 'event_state' and 'hometown_state' match: {state_matches}")

# Calculate the number of instances where 'event_country' and 'hometown_country' match but are not 'United States'
country_matches_excl_us = splits.apply(lambda row: (row['event_country'] == row['hometown_country']) and (row['event_country'] != 'United States'), axis=1).sum()

print(f"Number of instances where 'event_country' and 'hometown_country' match (excluding 'United States'): {country_matches_excl_us}")

city_matches = splits.apply(lambda row: row['event_city'] == row['training_city'], axis=1).sum()
print(f"Number of instances where 'event_city' and 'training_city' match: {city_matches}")

state_matches = splits.apply(lambda row: row['event_state'] == row['training_state'], axis=1).sum()
print(f"Number of instances where 'event_state' and 'training_state' match: {state_matches}")

country_matches_excl_us = splits.apply(lambda row: (row['event_country'] == row['training_country']) and (row['event_country'] != 'United States'), axis=1).sum()
print(f"Number of instances where 'event_country' and 'training_country' match (excluding 'United States'): {country_matches_excl_us}")


Number of instances where 'event_city' and 'hometown_city' match: 39
Number of instances where 'event_state' and 'hometown_state' match: 1091
Number of instances where 'event_country' and 'hometown_country' match (excluding 'United States'): 326
Number of instances where 'event_city' and 'training_city' match: 92
Number of instances where 'event_state' and 'training_state' match: 1062
Number of instances where 'event_country' and 'training_country' match (excluding 'United States'): 319


In [430]:
splits['result'] = splits['result'].replace({'win': 1, 'loss': 0, 'NC': 0, 'draw': 0})
splits

C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\651641542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  splits['result'] = splits['result'].replace({'win': 1, 'loss': 0, 'NC': 0, 'draw': 0})


,fighter name,hometown,training_location,result,win_method,win_details,location,event_city,event_state,event_country,training_city,training_state,training_country,hometown_city,hometown_state,hometown_country
1,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,1,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,Rio de Janeiro,None,Brazil,Manaus,None,Brazil
3,Tecia Torres,"Fall River, Massachusetts United States","Colorado Springs, Colorado United States",1,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,Colorado Springs,Colorado,United States,Fall River,Massachusetts,United States
7,Alonzo Menifield,"Los Angeles, California United States","Dallas, Texas United States",1,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,Dallas,Texas,United States,Los Angeles,California,United States
18,Chris Gruetzemacher,"Page, Arizona United States","New Bern, North Carolina United States",1,Decision,Unanimous,"UFC Apex, Las Vegas, Nevada, United States",Las Vegas,Nevada,United States,New Bern,North Carolina,United States,Page,Arizona,United States
26,Maycee Barber,"Greeley, Colorado United States","Fort Collins, Colorado United States",1,Decision,Split,"UFC Apex, Las Vegas, Nevada, United States",Las Vegas,Nevada,United States,Fort Collins,Colorado,United States,Greeley,Colorado,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19515,Daniel Pineda,"Dallas, TX USA","Houston, TX USA",0,Decision,Unanimous,"Central Pavilion Arena, Robstown, Texas, Unite...",Robstown,Texas,United States,Houston,TX,United States,Dallas,TX,United States
19573,Eddie Alvarez,"Philadelphia, Pennsylvania United States","Philadelphia, Pennsylvania United States",0,Decision,Unanimous,"Singapore Indoor Stadium, Singapore",Singapore Indoor Stadium,None,Singapore,Philadelphia,Pennsylvania,United States,Philadelphia,Pennsylvania,United States
19800,Hernani Perpetuo,Rio de Janeiro Brazil,Rio de Janeiro Brazil,0,Decision,Unanimous,"Ryogoku Kokugikan, Tokyo, Japan",Tokyo,None,Japan,Rio de Janeiro,None,Brazil,Rio de Janeiro,None,Brazil
19863,Yushin Okami,Kanagawa Japan,Kanagawa Japan,0,Decision,Unanimous,"Mall Of Asia Arena, Manila, Philippines",Manila,None,Philippines,Kanagawa,None,Japan,Kanagawa,None,Japan


In [431]:
# For event and hometown country match-up (excluding USA)
splits['event_hometown_country_match'] = ((splits['event_country'] == splits['hometown_country']) & (splits['event_country'] != 'United States')).astype(int)

# For event and hometown city match-up
splits['event_hometown_city_match'] = (splits['event_city'] == splits['hometown_city']).astype(int)

# For event and hometown state match-up
splits['event_hometown_state_match'] = (splits['event_state'] == splits['hometown_state']).astype(int)
# For event and training country match-up (excluding USA)
splits['event_training_country_match'] = ((splits['event_country'] == splits['training_country']) & (splits['event_country'] != 'United States')).astype(int)

# For event and training city match-up
splits['event_training_city_match'] = (splits['event_city'] == splits['training_city']).astype(int)

# For event and training state match-up
splits['event_training_state_match'] = (splits['event_state'] == splits['training_state']).astype(int)

splits

C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\939251820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  splits['event_hometown_country_match'] = ((splits['event_country'] == splits['hometown_country']) & (splits['event_country'] != 'United States')).astype(int)
C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\939251820.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  splits['event_hometown_city_match'] = (splits['event_city'] == splits['hometown_city']).astype(int)
C:\Users\tanne\AppData\Local\T

,fighter name,hometown,training_location,result,win_method,win_details,location,event_city,event_state,event_country,...,training_country,hometown_city,hometown_state,hometown_country,event_hometown_country_match,event_hometown_city_match,event_hometown_state_match,event_training_country_match,event_training_city_match,event_training_state_match
1,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,1,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,...,Brazil,Manaus,None,Brazil,0,0,0,0,0,0
3,Tecia Torres,"Fall River, Massachusetts United States","Colorado Springs, Colorado United States",1,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,...,United States,Fall River,Massachusetts,United States,0,0,0,0,0,0
7,Alonzo Menifield,"Los Angeles, California United States","Dallas, Texas United States",1,Decision,Unanimous,"Toyota Center, Houston, Texas, United States",Houston,Texas,United States,...,United States,Los Angeles,California,United States,0,0,0,0,0,1
18,Chris Gruetzemacher,"Page, Arizona United States","New Bern, North Carolina United States",1,Decision,Unanimous,"UFC Apex, Las Vegas, Nevada, United States",Las Vegas,Nevada,United States,...,United States,Page,Arizona,United States,0,0,0,0,0,0
26,Maycee Barber,"Greeley, Colorado United States","Fort Collins, Colorado United States",1,Decision,Split,"UFC Apex, Las Vegas, Nevada, United States",Las Vegas,Nevada,United States,...,United States,Greeley,Colorado,United States,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19515,Daniel Pineda,"Dallas, TX USA","Houston, TX USA",0,Decision,Unanimous,"Central Pavilion Arena, Robstown, Texas, Unite...",Robstown,Texas,United States,...,United States,Dallas,TX,United States,0,0,0,0,0,0
19573,Eddie Alvarez,"Philadelphia, Pennsylvania United States","Philadelphia, Pennsylvania United States",0,Decision,Unanimous,"Singapore Indoor Stadium, Singapore",Singapore Indoor Stadium,None,Singapore,...,United States,Philadelphia,Pennsylvania,United States,0,0,0,0,0,0
19800,Hernani Perpetuo,Rio de Janeiro Brazil,Rio de Janeiro Brazil,0,Decision,Unanimous,"Ryogoku Kokugikan, Tokyo, Japan",Tokyo,None,Japan,...,Brazil,Rio de Janeiro,None,Brazil,0,0,0,0,0,0
19863,Yushin Okami,Kanagawa Japan,Kanagawa Japan,0,Decision,Unanimous,"Mall Of Asia Arena, Manila, Philippines",Manila,None,Philippines,...,Japan,Kanagawa,None,Japan,0,0,0,0,0,0


In [439]:
import statsmodels.api as sm

# Define the predictor variables excluding 'hometown_state' and 'hometown_city'
predictors = ['event_hometown_city_match', 'event_hometown_state_match', 'event_hometown_country_match',
              'event_training_city_match', 'event_training_state_match', 'event_training_country_match']

Xtrain = splits[[ 'event_training_city_match', 'event_training_state_match', 'event_training_country_match']]
ytrain = splits[['result']] 

# Convert 'result' column to numeric (assuming it's not already)
splits['result'] = pd.to_numeric(splits['result'])

# building the model and fitting the data 
log_reg = sm.Logit(ytrain, Xtrain).fit() 

# Print the summary of the logistic regression model
print(log_reg.summary())


Optimization terminated successfully.
         Current function value: 0.690180
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 result   No. Observations:                 3777
Model:                          Logit   Df Residuals:                     3774
Method:                           MLE   Df Model:                            2
Date:                Wed, 10 Apr 2024   Pseudo R-squ.:               0.0007831
Time:                        18:57:22   Log-Likelihood:                -2606.8
converged:                       True   LL-Null:                       -2608.9
Covariance Type:            nonrobust   LLR p-value:                    0.1296
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
event_training_city_match       -0.1967      0.256     -0.768      0.442

C:\Users\tanne\AppData\Local\Temp\ipykernel_49436\3602069485.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  splits['result'] = pd.to_numeric(splits['result'])
